In [1]:
import pandas as pd
import numpy as np
import urllib.request
import cv2
import time

In [2]:
df = pd.read_csv('data/corpus.csv').drop(columns = 'Unnamed: 0')

In [3]:
df.head(1)

,index,user_identifier,description,date,longitude,latitude,external_upload_mean,camera,license_name,flickr_url,image_direct_url,user_tags,autotags
0,10000979206,66172503@N00,NaN,2013-09-30 00:26:09.0,14.499968,46.053249,"foursquare%3Avenue%3D4e2b1e5a315185636a08f0dd,...",NaN,Attribution License,http://www.flickr.com/photos/66172503@N00/1000...,http://farm4.staticflickr.com/3702/10000979206...,"instagram+app,iphoneography,nashville,square,s...","abstract:0.8090,architecture:0.9030,bright:0.6..."


In [4]:
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    try:
        resp = urllib.request.urlopen(url)
    except:
        return np.zeros((1,1))
    
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
    # return the image
    return image

In [59]:
def extract_infos(urls, max_stored = 1000):
    
    urls_todo = urls.values.tolist()
    colored = []
    
    while len(urls_todo) > 0:
        
        images = []
        
        # Scrap a first set of images
        for url in urls_todo[:max_stored]:
            images.append(url_to_image(url))
        
        # Remove non-found entries
        for i in range(len(images)):
            try:
                if images[i].shape[0] == 1:
                    colored.append(np.nan)
                    continue
            except:
                print('ERROR with image:', urls_todo[:max_stored][i], 'type', type(urls_todo[:max_stored][i]))
                colored.append(np.nan)
                continue
            
            # Color / b&w info
            if images[i].shape[2] == 3:
                colored.append(1)
            else:
                colored.append(0)            
        
        urls_todo = urls_todo[max_stored:]
        time.sleep(2)
                
    return colored
        

In [60]:
urls = df['image_direct_url'][:1000]

In [61]:
start = time.time()
colored = extract_infos(urls, max_stored = 250)
print(start - time.time())

ERROR with image: http://www.flickr.com/videos/87258464@N00/11589389194/play/orig/4c6a32727f . type <class 'str'>
-242.4047520160675


In [63]:
df.head(1)

,index,user_identifier,description,date,longitude,latitude,external_upload_mean,camera,license_name,flickr_url,image_direct_url,user_tags,autotags
0,10000979206,66172503@N00,NaN,2013-09-30 00:26:09.0,14.499968,46.053249,"foursquare%3Avenue%3D4e2b1e5a315185636a08f0dd,...",NaN,Attribution License,http://www.flickr.com/photos/66172503@N00/1000...,http://farm4.staticflickr.com/3702/10000979206...,"instagram+app,iphoneography,nashville,square,s...","abstract:0.8090,architecture:0.9030,bright:0.6..."
